# 🗒 Goal
The purpose of this notebook is to use GPT-4.0 Mini and the developed machine learning model to provide recommendations on clothing and activities for the user based on the weather data from the past 10 days, tailored to the user’s city of residence.

# Openai API Key set up

In [15]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# Load the developed model for temperature prediction

In [16]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import pickle
import json

# Paths
MODEL_PATH = "/content/drive/MyDrive/LLM/Temp_TimeSeries_LLM/gru_temperature_model.keras"
SCALER_PATH = "/content/drive/MyDrive/LLM/Temp_TimeSeries_LLM/temperature_scaler.pkl"
METADATA_PATH = "/content/drive/MyDrive/LLM/Temp_TimeSeries_LLM/temperature_metadata.json"

# Load GRU model
gru_model = tf.keras.models.load_model(MODEL_PATH)

# Load scaler
with open(SCALER_PATH, "rb") as f:
    scaler = pickle.load(f)

# Load metadata
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

past_history = metadata["past_history"]
target_col_index = metadata["target_col_index"]
feature_columns = metadata["feature_columns"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepare model input

In [17]:
import numpy as np
import pandas as pd

def prepare_model_input(last_10_days_df):
    """
    last_10_days_df: DataFrame with same feature columns as training data
    """
    # Ensure correct column order
    last_10_days_df = last_10_days_df[feature_columns]

    # Scale
    scaled = scaler.transform(last_10_days_df.values.astype(np.float32))

    # Add batch dimension → (1, timesteps, features)
    return np.expand_dims(scaled, axis=0)

# Predict in scaled version and return in °C

In [18]:
def predict_temperature(last_10_days_df):
    X = prepare_model_input(last_10_days_df)
    scaled_pred = gru_model.predict(X, verbose=0)[0][0]

    # Denormalize temperature
    t_mean = scaler.mean_[target_col_index]
    t_std = np.sqrt(scaler.var_[target_col_index])

    return float(scaled_pred * t_std + t_mean)

# GPT-4o-mini: activity, clothing & trend analysis

In [19]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [20]:
def gpt_recommendation(city, weather_summary, predicted_temp):
    """
    weather_summary: short text summary of last 10 days weather conditions
    """
    prompt = f"""
You are a helpful weather assistant.

City: {city}
Recent weather summary (last 10 days):
{weather_summary}

Predicted temperature for tomorrow (°C): {predicted_temp:.1f}

Tasks:
1. Describe the recent temperature trend in simple language.
2. Recommend suitable clothing based on the weather and the city of residence, considering fashion separately for men and women.
3. Suggest appropriate daily activities by considering weather and the city of residence.

Keep the response balanced (not too long, not too short), friendly, and easy to understand.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a friendly weather advisor."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6
    )

    return response.choices[0].message.content

# Gradio interface logic

In [21]:
def gradio_app(city, uploaded_file):
    # Immediately show typing / thinking message
    yield "⌛ **AI is analyzing the weather data and preparing recommendations…**"

    import pandas as pd

    if uploaded_file is None:
        yield "❌ Please upload a CSV file."
        return

    # Read CSV
    try:
        df_input = pd.read_csv(uploaded_file)
    except Exception as e:
        yield f"❌ Failed to read CSV file: {e}"
        return

    # Optional: handle date column
    if "Date" in df_input.columns:
        df_input = df_input.sort_values("Date")
        df_input = df_input.drop(columns=["Date"])

    # Validation
    if df_input.shape[0] != past_history:
        yield f"❌ CSV must contain exactly {past_history} rows."
        return

    missing_cols = set(feature_columns) - set(df_input.columns)
    if missing_cols:
        yield f"❌ Missing required columns: {missing_cols}"
        return

    # Ensure correct column order
    df_input = df_input[feature_columns]

    # Prediction
    pred_temp = predict_temperature(df_input)

    # Weather summary for GPT
    temp_series = df_input["T (degC)"].tolist()
    weather_summary = (
        f"Temperature ranged from {min(temp_series):.1f}°C to {max(temp_series):.1f}°C.\n"
        f"Average temperature: {sum(temp_series)/len(temp_series):.1f}°C."
    )

    gpt_text = gpt_recommendation(city, weather_summary, pred_temp)

    # Final output
    yield f"""
🌡 **Predicted Temperature (Tomorrow):** {pred_temp:.2f} °C

🧠 **Weather Insights & Recommendations**
{gpt_text}
"""

# Build and launch Gradio app

In [22]:
import gradio as gr

interface = gr.Interface(
    fn=gradio_app,
    inputs=[
        gr.Textbox(label="City Name", placeholder="e.g., Berlin"),
        gr.File(
            label="Upload CSV: Last 10 Days Weather Data",
            file_types=[".csv"]
        )
    ],
    outputs=gr.Markdown(),
    title="🌤️ AI Weather Forecast & Lifestyle Advisor",
    description=(
        "Upload a CSV file containing the last 10 days of weather data "
        "(all required parameters). The model predicts tomorrow’s temperature "
        "and provides clothing & activity recommendations using AI."
    )
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b97826c918d563d3c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
